In [4]:
#SETTING PATH 
path="/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
previousResults="/sharedFolder/Results/Hotspot/Revision1/FULL/"
results="/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"
dir.create(results,recursive=TRUE)
riderPath="/home/Riderv2/"

In [5]:
#LOAD USEFULL FUNCTIONS 
noExt=function(x){
return(tools::file_path_sans_ext(x))
}
bnoExt=function(x){
return(basename(tools::file_path_sans_ext(x)))
}
fNumber=function(x){
sapply(x,FUN=function(x){
strsplit(x,"__")[[1]][2]
})
}
rRider=function(x){
sapply(x,FUN=function(x){
strsplit(x,"-RIDER")[[1]][1]
})
}
# centering with 'scale()'
center_scale <- function(x) {
    scale(x, scale = FALSE)
}

In [6]:
#LOAD LIBRARIES 
library("argparser");
library(gtools);
library("GenomicRanges");
library("ggVennDiagram");
options(warn=-1)
library(GenomicRanges)

In [9]:
library(gtools)
library(GenomicRanges)

# Funzioni utili
noExt <- function(x) tools::file_path_sans_ext(x)
fNumber <- function(x) strsplit(x, "__")[[1]][2]

# Path fissi
path <- "/sharedFolder/Data/1_HTGTS/13_Revision1/pooled/"
base_results <- "/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/"
dir.create(base_results, recursive = TRUE, showWarnings = FALSE)

thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {

  message("\n--- Processing threshold: ", jamesonFirst, " ---")
  
  # Percorsi
  previousResults_thr <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst)
  results_thr <- paste0(base_results, jamesonFirst, "/")
  dir.create(results_thr, recursive = TRUE, showWarnings = FALSE)

  # Carica e filtra il database
  db_path <- paste0(previousResults_thr, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  Database <- Database[grep("Idelalisib", Database$Sample), ]
  
  # Filtra i .bed di Idelalisib
  a <- list.files(path, pattern = ".bed", recursive = TRUE)
  a <- mixedsort(a)
  a2 <- sapply(a, function(x) strsplit(x, ".bed")[[1]][1])
  a2 <- sapply(a2, function(x) strsplit(x, "__")[[1]][2])
  idx <- grep("Idelalisib", a2)
  a <- a[idx]
  a2 <- a2[idx]

  # Inizializzazione matrici
  hotspots <- unique(Database$hotspotID)
  hgtgts_count <- matrix(0, ncol = length(hotspots), nrow = length(a2))
  colnames(hgtgts_count) <- hotspots
  rownames(hgtgts_count) <- a2
  hgtgts_rpm <- hgtgts_count

  # Conta overlaps
  for (j in a) {
    message(" > File: ", j)
    temp <- read.table(paste0(path, "/", j), sep = "\t")
    tempGG <- GRanges(seqnames = temp[,1], ranges = IRanges(start = as.numeric(temp[,2]), end = as.numeric(temp[,3])))
    name <- as.vector(fNumber(j))
    
    for (i in seq_len(nrow(Database))) {
      gg <- GRanges(seqnames = Database[i,1], ranges = IRanges(start = as.numeric(Database[i,2]), end = as.numeric(Database[i,3])))
      over <- countOverlaps(gg, tempGG)
      hgtgts_count[name, toString(Database[i,7])] <- over
      hgtgts_rpm[name, toString(Database[i,7])] <- (over / nrow(temp)) * 1e6
    }
  }

  # Scrivi file "plain"
  write.table(t(hgtgts_count), paste0(results_thr, "count_hgtgtsHeatmap.csv"), col.names = NA, sep = ",")
  write.table(t(hgtgts_rpm), paste0(results_thr, "rpm_hgtgtsHeatmap.csv"), col.names = NA, sep = ",")

  # Annotazioni
  annotations <- c()
  for (n in colnames(hgtgts_count)) {
    ann <- unique(Database[which(Database$hotspotID == n), ]$annotation)
    if (is.null(ann) || ann == "NULL") {
      ann <- paste(unique(Database[which(Database$hotspotID == n), ]$Chromosome), ":",
                   unique(Database[which(Database$hotspotID == n), ]$ChrStart), "-",
                   unique(Database[which(Database$hotspotID == n), ]$ChrEnd), sep = "")
    }
    annotations <- c(annotations, ann)
  }

  colnames(hgtgts_count) <- annotations
  colnames(hgtgts_rpm) <- annotations

  # Scrivi file ANNOTATED
  write.table(t(hgtgts_count), paste0(results_thr, "count_hgtgtsHeatmapANNOTATED.csv"), col.names = NA, sep = ",")
  write.table(t(hgtgts_rpm), paste0(results_thr, "rpm_hgtgtsHeatmapANNOTATED.csv"), col.names = NA, sep = ",")
}



--- Processing threshold: 5 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed


--- Processing threshold: 6 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed


--- Processing threshold: 7 ---

 > File: 7__MEC-1_Idelalisib_DMSO.bed

 > File: 8__MEC-1_Idelalisib_Tazemetostat.bed

 > File: 9__MEC-1_Idelalisib_Valemetostat.bed



In [16]:
library(eulerr)
library(grid)
library(gtools)

types <- c("Idelalisib")
conditions <- c("DMSO", "Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  
  for (j in types) {
    subBase=Database[grep(j,Database[,6]),]
    x <- list()
    for (i in conditions) {
    x[[i]]=subBase[grep(i,subBase[,6]),7]
    }
    
    venn_counts <- c(
      "DMSO" = length(setdiff(setdiff(x$DMSO, x$Tazemetostat), x$Valemetostat)),
      "Tazemetostat" = length(setdiff(setdiff(x$Tazemetostat, x$DMSO), x$Valemetostat)),
      "Valemetostat" = length(setdiff(setdiff(x$Valemetostat, x$Tazemetostat), x$DMSO)),
      "DMSO&Tazemetostat&Valemetostat" = length(Reduce(intersect, x)),
      "DMSO&Tazemetostat" = length(setdiff(intersect(x$DMSO, x$Tazemetostat), x$Valemetostat)),
      "DMSO&Valemetostat" = length(setdiff(intersect(x$DMSO, x$Valemetostat), x$Tazemetostat)),
      "Tazemetostat&Valemetostat" = length(setdiff(intersect(x$Tazemetostat, x$Valemetostat), x$DMSO))
    )
    
    fit <- euler(venn_counts)
    
    # Output path
    out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
    dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)
    
    # PDF device con grid-compatible output
    cairo_pdf(filename = paste0(out_dir, "venn_diagram.pdf"), width = 6, height = 6)
    grid.draw(plot(fit, quantities = TRUE, main = paste("Idelalisib Hotspot - Threshold", jamesonFirst)))
    dev.off()
  }
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [18]:
library(eulerr)
library(grid)

conditions <- c("DMSO", "Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Load and filter database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  
  subBase <- Database[grep("Idelalisib", Database[,6]), ]  # Usa correttamente la colonna 6

  # Costruzione delle liste per le condizioni
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  # Sezioni di interesse
  DMSOSpec   <- setdiff(setdiff(x$DMSO, x$Valemetostat), x$Tazemetostat)
  TazeSpec   <- setdiff(setdiff(x$Tazemetostat, x$DMSO), x$Valemetostat)
  ValeSpec   <- setdiff(setdiff(x$Valemetostat, x$DMSO), x$Tazemetostat)
  TazVale    <- setdiff(intersect(x$Tazemetostat, x$Valemetostat), x$DMSO)
  TazDMSO    <- setdiff(intersect(x$Tazemetostat, x$DMSO), x$Valemetostat)
  DMSOVale   <- setdiff(intersect(x$Valemetostat, x$DMSO), x$Tazemetostat)
  common     <- Reduce(intersect, x)

  # Venn diagram tra DMSO e Tazemetostat
  fit1 <- euler(c(
    "DMSO" = length(setdiff(x$DMSO, x$Tazemetostat)),
    "Tazemetostat" = length(setdiff(x$Tazemetostat, x$DMSO)),
    "DMSO&Tazemetostat" = length(intersect(x$DMSO, x$Tazemetostat))
  ))

  # Path risultati
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  # Salva Venn diagram
  cairo_pdf(paste0(out_dir, "venn_dmsovstaze.pdf"), width = 6, height = 6)
  grid.draw(plot(fit1, quantities = TRUE, main = paste("Idelalisib Hotspot - DMSO vs Tazemetostat - Threshold", jamesonFirst)))
  dev.off()

  # Scrivi i file delle liste
  res2 <- paste0(out_dir, "HotspotIDList/")
  dir.create(res2, showWarnings = FALSE)

  write.table(unique(union(x$Tazemetostat, x$DMSO)), paste0(res2, "TazeFULL.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(unique(union(x$Valemetostat, x$DMSO)), paste0(res2, "ValeFULL.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(intersect(x$Tazemetostat, x$DMSO), paste0(res2, "Tazecommon.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
  write.table(intersect(x$Valemetostat, x$DMSO), paste0(res2, "Valecommon.csv"), col.names = FALSE, row.names = FALSE, quote = FALSE)
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [20]:
library(eulerr)
library(grid)

conditions <- c("Tazemetostat", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Load and filter database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  subBase <- Database[grep("Idelalisib", Database[,6]), ]
  
  # Costruzione liste
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  # Per comodità di sintassi
  j2 <- x
  
  # Costruzione eulerr
  fit2 <- euler(c(
    "Tazemetostat" = length(setdiff(j2$Tazemetostat, j2$Valemetostat)),
    "Valemetostat" = length(setdiff(j2$Valemetostat, j2$Tazemetostat)),
    "Tazemetostat&Valemetostat" = length(intersect(j2$Tazemetostat, j2$Valemetostat))
  ))

  # Salvataggio
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  cairo_pdf(paste0(out_dir, "venn_tazevsvale.pdf"), width = 6, height = 6)
  grid.draw(plot(fit2, quantities = TRUE, main = paste("Idelalisib Hotspot - Taze vs Vale - Threshold", jamesonFirst)))
  dev.off()
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---


In [21]:
library(eulerr)
library(grid)

conditions <- c("DMSO", "Valemetostat")
thresholds <- c(5, 6, 7)

for (jamesonFirst in thresholds) {
  cat("\n--- Processing threshold:", jamesonFirst, "---\n")
  
  # Carica e filtra il database
  db_path <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/", jamesonFirst, "/Database_annotated.csv")
  Database <- read.table(db_path, header = TRUE, sep = ",")
  Database <- Database[which(Database$Category == "AID_off_target"), ]
  subBase <- Database[grep("Idelalisib", Database[,6]), ]

  # Crea le liste hotspot
  x <- list()
  for (cond in conditions) {
    x[[cond]] <- subBase[grep(cond, subBase[,6]), "hotspotID"]
  }

  j2 <- x  # alias

  # Costruisci Venn diagram
  fit3 <- euler(c(
    "DMSO" = length(setdiff(j2$DMSO, j2$Valemetostat)),
    "Valemetostat" = length(setdiff(j2$Valemetostat, j2$DMSO)),
    "DMSO&Valemetostat" = length(intersect(j2$DMSO, j2$Valemetostat))
  ))

  # Salva il plot
  out_dir <- paste0("/sharedFolder/Results/Hotspot/Revision1/FULL/Idelalisib/", jamesonFirst, "/")
  dir.create(out_dir, recursive = TRUE, showWarnings = FALSE)

  cairo_pdf(paste0(out_dir, "venn_dmsovsvale.pdf"), width = 6, height = 6)
  grid.draw(plot(fit3, quantities = TRUE, main = paste("Idelalisib Hotspot - DMSO vs Vale - Threshold", jamesonFirst)))
  dev.off()
}



--- Processing threshold: 5 ---

--- Processing threshold: 6 ---

--- Processing threshold: 7 ---
